In [ ]:
import cocpit.pic as pic
import cocpit
import pandas as pd
import matplotlib.pyplot as plt
import os
from collections import defaultdict
%load_ext autoreload
%autoreload 2

In [ ]:
campaign='ARM'
df = pd.read_csv('final_databases/no_mask/'+campaign+'.csv')
desired_size = 1000
for file, class_ in zip(df['filename'], df['classification']):
    print(file)
    image = cocpit.pic.Image('cpi_data/campaigns/'+campaign+'/single_imgs/', file)
    image.resize_stretch(desired_size)
    print(class_)
    if class_ == 'compact irregular':
        image.create_ellipse()
        plt.imshow(image.image_og)
        plt.show()